In [53]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [54]:
# loading dataset
df_f = pd.read_csv('../TLAB 2 PHASE 2/newdata.csv')

randomsample= df_f.sample(n=500000)

In [55]:
randomsample

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,PAYMENT,CASH_IN,CASH_OUT,DEBIT,TRANSFER
38011,9,PAYMENT,1643.74,0.00,0.00,0.00,0.00,0,1,0,0,0,0
3893132,284,CASH_IN,218163.48,2959330.15,3177493.63,2033687.30,1815523.81,0,0,1,0,0,0
720931,37,CASH_IN,14215.78,2785422.27,2799638.06,3830411.14,3816195.36,0,0,1,0,0,0
4262461,306,CASH_OUT,126380.89,0.00,0.00,4046588.93,4172969.82,0,0,0,1,0,0
1820524,163,PAYMENT,9934.11,50808.00,40873.89,0.00,0.00,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609620,156,TRANSFER,720477.51,40947.69,0.00,3867350.18,4587827.69,0,0,0,0,0,1
4065148,300,CASH_OUT,118288.57,0.00,0.00,306776.49,425065.06,0,0,0,1,0,0
2948241,230,CASH_IN,341506.74,10306.00,351812.74,0.00,0.00,0,0,1,0,0,0
1520208,153,CASH_IN,86493.06,578650.00,665143.06,385777.28,299284.22,0,0,1,0,0,0


In [94]:
# type is non-integer, so we must drop it.

#randomsample = randomsample.drop(['type'], axis= 1)

randomsample.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,PAYMENT,CASH_IN,CASH_OUT,DEBIT,TRANSFER
38011,9,1643.74,0.00,0.00,0.00,0.00,0,1,0,0,0,0
3893132,284,218163.48,2959330.15,3177493.63,2033687.30,1815523.81,0,0,1,0,0,0
720931,37,14215.78,2785422.27,2799638.06,3830411.14,3816195.36,0,0,1,0,0,0
4262461,306,126380.89,0.00,0.00,4046588.93,4172969.82,0,0,0,1,0,0
1820524,163,9934.11,50808.00,40873.89,0.00,0.00,0,1,0,0,0,0


In [64]:
#Define target data
y= randomsample['isFraud']

#Loading the X variables
x= randomsample.drop(['isFraud'], axis= 1)

In [65]:
#Train and Test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.20,random_state = 42)

In [66]:
print(f'x_train : {x_train.shape}')
print(f'y_train : {y_train.shape}')
print(f'x_test : {x_test.shape}')
print(f'y_test: {y_test.shape}')

x_train : (400000, 11)
y_train : (400000,)
x_test : (100000, 11)
y_test: (100000,)


In [67]:
rf_Model = RandomForestClassifier()

In [68]:
#  Fitting the model
rf_Model.fit(x_train,y_train)

RandomForestClassifier()

In [69]:
print(f'Train Accuracy = {rf_Model.score(x_train,y_train):.3f}')
print(f'Test Accuracy = {rf_Model.score(x_test,y_test):.3f}')

Train Accuracy = 1.000
Test Accuracy = 1.000


In [73]:
from sklearn.metrics import f1_score
y_pred = rf_Model.predict(x_test)
f1 = f1_score(y_test, y_pred)
print("F1 Score:",f1)

F1 Score: 0.8539325842696629


In [74]:
# Since the test accuracy is reasonably high and close to the train accuracy, it suggests that the model is performing well and generalizing effectively. But 100% in this case means overfiitting and it will perform poorly on truly unseen data.

In [90]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of level in tree
max_depth = [int(x)for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

In [95]:
# Use the random grid to search tor best hyperparameters
# Random search of parameters, using 3 fold cross validation.
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_Model, param_distributions = random_grid, n_iter=10, cv=2, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
rf_random.fit(x_train,y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END bootstrap=True, max_depth=50, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time= 2.9min
[CV] END bootstrap=True, max_depth=50, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time= 3.1min
[CV] END bootstrap=False, max_depth=90, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 8.2min
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 8.3min
[CV] END bootstrap=False, max_depth=90, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 8.7min
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 8.8min
[CV] END bootstrap=True, max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=140

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [96]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': False}

In [97]:
print(f'Train Accuracy = {rf_random.score(x_train,y_train):.3}')
print(f'Test Accuracy = {rf_random.score(x_test,y_test):.3f}')

Train Accuracy = 1.0
Test Accuracy = 1.000


In [100]:
from sklearn.metrics import f1_score
y_pred = rf_random. predict(x_test)
f1 = f1_score(y_test, y_pred)
print ("F1 Score:",f1)

F1 Score: 0.8507462686567163
